In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
#dataset_dir = '/home/mazhar/FaceMaskDetection-jupyter/data'
# dataset download link -> https://www.kaggle.com/omkargurav/face-mask-dataset
dataset_dir = ''

In [3]:
BATCH_SIZE = 128
IMG_SIZE = (160, 160)

In [4]:
train_dataset = image_dataset_from_directory(dataset_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 7553 files belonging to 2 classes.


In [5]:
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [6]:
train_dataset, validation_dataset, test_dataset = get_dataset_partitions_tf(train_dataset, 
                                                                     len(train_dataset))

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [21]:
'''data_augmentor = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.3)
])

for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentor(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')'''

"data_augmentor = tf.keras.Sequential([\n    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),\n    tf.keras.layers.experimental.preprocessing.RandomRotation(0.3)\n])\n\nfor image, _ in train_dataset.take(1):\n    plt.figure(figsize=(10, 10))\n    first_image = image[0]\n    for i in range(9):\n        ax = plt.subplot(3, 3, i + 1)\n        augmented_image = data_augmentor(tf.expand_dims(first_image, 0))\n        plt.imshow(augmented_image[0] / 255)\n        plt.axis('off')"

In [8]:
from tensorflow.keras.applications import mobilenet_v2, MobileNetV2
input_preprocessor = mobilenet_v2.preprocess_input

In [9]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = MobileNetV2(input_shape = IMG_SHAPE,
                        include_top=False)

base_model.trainable = False

In [10]:
# Model

inputs = tf.keras.Input(shape=IMG_SHAPE)
X = input_preprocessor(inputs)
X = base_model(X)
X = tf.keras.layers.GlobalAveragePooling2D()(X)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(X)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [11]:
class MyEarlyStopping(tf.keras.callbacks.Callback):
    
    def __init__(self, monitor='val_accuracy', baseline=0.99):
        super(MyEarlyStopping, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        loss = logs.get(self.monitor)
        if loss is not None:
            if loss >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

EarylyStop = MyEarlyStopping(monitor='val_accuracy', baseline=0.99)

In [12]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
             metrics=['accuracy'])

In [13]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_______________________________________________

In [28]:
history = model.fit(train_dataset,
                    epochs=5,
                    validation_data=validation_dataset, callbacks=[EarylyStop])

Epoch 1/5


2021-07-31 22:46:37.639343: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-31 22:46:37.672643: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2693820000 Hz


48/48 [==============================] - 81s 1s/step - loss: 0.2089 - accuracy: 0.9304 - val_loss: 0.0734 - val_accuracy: 0.9870
Epoch 2/5
48/48 [==============================] - 83s 2s/step - loss: 0.0545 - accuracy: 0.9865 - val_loss: 0.0467 - val_accuracy: 0.9883
Epoch 3/5
48/48 [==============================] - 100s 2s/step - loss: 0.0348 - accuracy: 0.9899 - val_loss: 0.0255 - val_accuracy: 0.9922
Epoch 2: Reached baseline, terminating training


In [29]:
model.evaluate(test_dataset)

6/6 [==============================] - 15s 1s/step - loss: 0.0419 - accuracy: 0.9857


[0.041862089186906815, 0.9856770634651184]

In [30]:
model.save('models/face_mask_model.h5')

/home/mazhar/.local/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
